# Predictive Credit Risk Modeling: Leveraging Decision Trees, XGBoost, and Random Forests
## Zulfikar Azaria Rahman	ID/X Partners (Rakamin Academy)


# Import Library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    f1_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
    accuracy_score,
    ConfusionMatrixDisplay,
    auc,
    precision_score,
    recall_score,
)
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Suppress warnings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore")

# Import Data

In [ ]:
data_raw = pd.read_csv('loan_data_2007_2014.csv')

In [ ]:
data_raw.info()

#  Preliminary Look

In [ ]:
data_raw.info()

In [ ]:
data_raw.sample(5)

There are features that are not useful, hence the need to remove features. Examples include features that are unique id, free text, empty values.

In [ ]:
col_to_drop = [
    'Unnamed: 0' 
    ,'id'
    , 'member_id'
    , 'url'
    , 'desc'
    , 'zip_code' 
    , 'annual_inc_joint'
    , 'dti_joint'
    , 'verification_status_joint'
    , 'open_acc_6m'
    , 'open_il_6m'
    , 'open_il_12m'
    , 'open_il_24m'
    , 'mths_since_rcnt_il'
    , 'total_bal_il'
    , 'il_util'
    , 'open_rv_12m'
    , 'open_rv_24m'
    , 'max_bal_bc'
    , 'all_util'
    , 'inq_fi'
    , 'total_cu_tl'
    , 'inq_last_12m'
]

In [ ]:
data = data_raw.drop(col_to_drop, axis=1)

In [ ]:
data.head()

## Define Target Variables

In project credit risk modelling, the main objective is to predict an individual's ability to repay the loan. Therefore, the target variable used should reflect the individual's ability in this regard. In this data set, the variable `loan_status` is a variable that can be used as a target variable because it reflects the performance of each individual in making payments on loans so far.

In [ ]:
data.loan_status.value_counts(normalize=True)*100

In [ ]:
bad_debt = [
    'Charged Off' 
    , 'Default' 
    , 'Does not meet the credit policy. Status:Charged Off'
    , 'Late (31-120 days)'
]

data['credit_status'] = np.where(data['loan_status'].isin(bad_debt), 1, 0)

In [ ]:
data['credit_status'].value_counts(normalize=True)*100

In [ ]:
data.drop('loan_status', axis=1, inplace=True)

# Data Cleaning & Data Preprocessing

`term`

In [ ]:
data['term'].unique()

In [ ]:
data['term'] = data['term'].str.replace(' months', '').astype(float)

In [ ]:
data['term'].unique()

`emp_length`

In [ ]:
data['emp_length'].unique()

In [ ]:
data['emp_length'] = data['emp_length'].str.replace('+ years', '')
data['emp_length'] = data['emp_length'].str.replace('< 1 year', str(0))
data['emp_length'] = data['emp_length'].str.replace(' years', '')
data['emp_length'] = data['emp_length'].str.replace(' year', '')
data['emp_length'] = data['emp_length'].astype(float)

`earliest_cr_line`

Modifies `earliest_cr_line` from month-year format to a calculation of how much time has passed since that time. To do this I use 2016-12-01 as reference date.

In [ ]:
reference_date = '2016-12-01'

In [ ]:
data['earliest_cr_line_date'] = pd.to_datetime(data['earliest_cr_line'], format='%b-%y')
data['earliest_cr_line_date'].head()

In [ ]:
data['mths_since_earliest_cr_line'] = round(pd.to_numeric((pd.to_datetime(reference_date) - data['earliest_cr_line_date']) / np.timedelta64(1, 'm')))
data['mths_since_earliest_cr_line'].head()

In [ ]:
data.mths_since_earliest_cr_line.describe()

There is a strange value, which is negative.

In [ ]:
data[data['mths_since_earliest_cr_line']<0][['earliest_cr_line', 'earliest_cr_line_date', 'mths_since_earliest_cr_line']].head()

It turns out that the negative value is because the Python function misinterpreted the year 62 to be 2062, when it should have been 1962.
To solve this I simply changed the negative value to the maximum value of the feature (`mths_since_earliest_cr_line`).

In [ ]:
data.loc[data['mths_since_earliest_cr_line']<0, 'mths_since_earliest_cr_line'] = data['mths_since_earliest_cr_line'].max()
data.drop(['earliest_cr_line', 'earliest_cr_line_date'], axis=1, inplace=True)

 `issue_d`

In [ ]:
data['issue_d_date'] = pd.to_datetime(data['issue_d'], format='%b-%y')
data['mths_since_issue_d'] = round(pd.to_numeric((pd.to_datetime(reference_date) - data['issue_d_date']) / np.timedelta64(1, 'm')))

In [ ]:
data['mths_since_issue_d'].describe()

In [ ]:
data.drop(['issue_d', 'issue_d_date'], axis=1, inplace=True)

`last_pymnt_d`

In [ ]:
data['last_pymnt_d_date'] = pd.to_datetime(data['last_pymnt_d'], format='%b-%y')
data['mths_since_last_pymnt_d'] = round(pd.to_numeric((pd.to_datetime(reference_date) - data['last_pymnt_d_date']) / np.timedelta64(1, 'm')))

In [ ]:
data['mths_since_last_pymnt_d'].describe()

In [ ]:
data.drop(['last_pymnt_d', 'last_pymnt_d_date'], axis=1, inplace=True)

`next_pymnt_d`

In [ ]:
data['next_pymnt_d_date'] = pd.to_datetime(data['next_pymnt_d'], format='%b-%y')
data['mths_since_next_pymnt_d'] = round(pd.to_numeric((pd.to_datetime(reference_date) - data['next_pymnt_d_date']) / np.timedelta64(1, 'm')))

In [ ]:
data.drop(['next_pymnt_d', 'next_pymnt_d_date'], axis=1, inplace=True)

`last_credit_pull_d`

In [ ]:
data['last_credit_pull_d_date'] = pd.to_datetime(data['last_credit_pull_d'], format='%b-%y')
data['mths_since_last_credit_pull_d'] = round(pd.to_numeric((pd.to_datetime(reference_date) - data['last_credit_pull_d_date']) / np.timedelta64(1, 'm')))

In [ ]:
data['mths_since_last_credit_pull_d'].describe()

In [ ]:
data.drop(['last_credit_pull_d', 'last_credit_pull_d_date'], axis=1, inplace=True)

In [ ]:
# separating numerical and categorical features
nums = []
cats = []
for i in data.columns:
  if data[i].dtype == 'object':
    cats.append(i)
  else:
    nums.append(i)
print('jumlah = ',len(nums))
print('nums = ',nums)
print('jumlah = ',len(cats))
print('cats = ',cats)

## Statistical Summary For Numericals Columns

In [ ]:
# numerical statistical sumary
data[nums].describe().T

## Statistical Summary For Categorical Columns

In [ ]:
# numerical statistical sumary
data[cats].describe().T

In [ ]:
# separating numerical and categorical features
nums = []
cats = []
for i in data.columns:
  if data[i].dtype == 'object':
    cats.append(i)
  else:
    nums.append(i)

In [ ]:
for col in data[cats]:
  print(f"Value counts of {col} column")
  print(data[col].value_counts(), '\n')

Observation:
1. Drop `application_type`, `policy_code` because it only has 1 unique value


In [ ]:
data.drop(['application_type', 'policy_code', 'sub_grade'], axis=1, inplace=True)

# Missing Value Check

In [ ]:
# Checking for missing value
Missing = data.isnull().sum() * 100 / len(data)
dtypes=[data[col].dtype for col in data.columns]
Missing_Value = pd.DataFrame({'data_type':dtypes,
                                 'Missing': Missing})
Missing_Value.sort_values('Missing', ascending=False, inplace=True)
Missing_Value

Because the missing value of `mths_since_last_record` is more than 60%, so I drop the `mths_since_last_record` column. 

In [ ]:
data.drop(['mths_since_last_record', 'mths_since_last_major_derog', 'mths_since_last_delinq'], axis=1, inplace=True)

In [ ]:
# Select only numeric columns
numeric_data = data.select_dtypes(include=['number'])

corr = numeric_data.corr()

plt.figure(figsize=(20, 20))
heatmap = sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', linewidths=0.1, annot_kws={"size": 8})

# Reduce the font size of x and y axis labels
heatmap.set_xticklabels(heatmap.get_xticklabels(), fontsize=10)
heatmap.set_yticklabels(heatmap.get_yticklabels(), fontsize=10)

plt.title('Correlation Heatmap', fontsize=16)
plt.show()

In [ ]:
# Calculate the correlation matrix and take its absolute value
corr_matrix = numeric_data.corr().abs()

# Taking the top of the correlation matrix (above the diagonal)
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find columns with high correlation (>0.7)
drop_hicorr = [column for column in upper.columns if any(upper[column] > 0.7)]

In [ ]:
data.drop(['funded_amnt',
 'funded_amnt_inv',
 'installment',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'mths_since_last_pymnt_d',
 'mths_since_next_pymnt_d',
 'mths_since_last_credit_pull_d'], axis=1, inplace=True)

In [ ]:
# Checking for missing value
Missing = data.isnull().sum() * 100 / len(data)
dtypes=[data[col].dtype for col in data.columns]
Missing_Value = pd.DataFrame({'data_type':dtypes,
                                 'Missing': Missing})
Missing_Value.sort_values('Missing', ascending=False, inplace=True)
Missing_Value

## Handling Missing Value 

In [ ]:
data['tot_cur_bal'].fillna(0, inplace=True)
data['tot_coll_amt'].fillna(0, inplace=True)
data['emp_length'].fillna(0, inplace=True)
data['revol_util'].fillna(0, inplace=True)
data['collections_12_mths_ex_med'].fillna(0, inplace=True)
data['open_acc'].fillna(0, inplace=True)
data['delinq_2yrs'].fillna(0, inplace=True)
data['pub_rec'].fillna(0, inplace=True)
data['inq_last_6mths'].fillna(0, inplace=True)
data['total_acc'].fillna(0, inplace=True)
data['acc_now_delinq'].fillna(0, inplace=True)
data['mths_since_earliest_cr_line'].fillna(0, inplace=True)
data['annual_inc'].fillna(data['annual_inc'].mean(), inplace=True)
data['total_rev_hi_lim'].fillna(data['total_rev_hi_lim'].median(), inplace=True)

In [ ]:
#drop rows that has missing values
data.dropna(inplace=True)

In [ ]:
# Sanity Checking for missing value
Missing = data.isnull().sum() * 100 / len(data)
dtypes=[data[col].dtype for col in data.columns]
Missing_Value = pd.DataFrame({'data_type':dtypes,
                                 'Missing': Missing})
Missing_Value.sort_values('Missing', ascending=False, inplace=True)
Missing_Value

## Duplicated data check

In [ ]:
# number of duplicated data
data.duplicated().sum()

Data is clean now

# Data Transformation & Standardrization

## Label Encode

In [ ]:
cat_LE = [col for col in data.select_dtypes(include='object').columns.tolist()]

In [ ]:
cat_LE

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for i in cat_LE:
   data[i] = le.fit_transform(data[i])
data.sample(5)

In [ ]:
# Select only numeric columns
numeric_data = data.select_dtypes(include=['number'])

corr = numeric_data.corr()

plt.figure(figsize=(20, 20))
heatmap = sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', linewidths=0.1, annot_kws={"size": 8})

# Reduce the font size of x and y axis labels
heatmap.set_xticklabels(heatmap.get_xticklabels(), fontsize=10)
heatmap.set_yticklabels(heatmap.get_yticklabels(), fontsize=10)

plt.title('Correlation Heatmap', fontsize=16)
plt.show()


# Modeling

## Train Test Spling

In [ ]:
data.sample(5)

In [ ]:
feature = data.drop('credit_status', axis=1)
target = data['credit_status']

In [ ]:
# First splitting: pretrain and test
feature_pretrain, feature_test, target_pretrain, target_test = train_test_split(feature, target, test_size=0.20, random_state=42)

# Second splitting: train and validation
feature_train, feature_validation, target_train, target_validation = train_test_split(feature_pretrain, target_pretrain, test_size=0.20, random_state=42)

In [ ]:
# Initialize StandardScaler
scaler = StandardScaler()

# Fit and transform the training data
feature_train_scaled = scaler.fit_transform(feature_train)

# Transform the validation and test data
feature_validation_scaled = scaler.transform(feature_validation)
feature_test_scaled = scaler.transform(feature_test)


In [ ]:
# Perform oversampling with SMOTE on the scaled training data
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(feature_train_scaled, target_train)

# Check the number of samples before and after oversampling
print("Jumlah sampel sebelum oversampling:", len(feature_train))
print("Jumlah sampel setelah oversampling:", len(X_resampled))

In [ ]:
Model = [
 ('XGBoost', XGBClassifier()),
 ('Decision Tree', DecisionTreeClassifier(random_state=42)),
 ('Random Forest', RandomForestClassifier(random_state=42))
]

In [ ]:
results_train = []

for name, model in Model:
    model.fit(X_resampled, y_resampled)
    
    # Prediction and evaluation on validation data
    y_pred_train_model = model.predict(feature_validation_scaled)
    accuracy_train = accuracy_score(target_validation, y_pred_train_model)
    precision_train = precision_score(target_validation, y_pred_train_model)
    recall_train = recall_score(target_validation, y_pred_train_model)
    f1_train = f1_score(target_validation, y_pred_train_model)
    
    # Prediction and evaluation on test data
    y_pred_test_model = model.predict(feature_test_scaled)
    accuracy_test = accuracy_score(target_test, y_pred_test_model)
    precision_test = precision_score(target_test, y_pred_test_model)
    recall_test = recall_score(target_test, y_pred_test_model)
    f1_test = f1_score(target_test, y_pred_test_model)
    
    results_train.append((name, accuracy_train, precision_train, recall_train, f1_train, accuracy_test, precision_test, recall_test, f1_test))

# Create a DataFrame from the result
comparison_df_train = pd.DataFrame(results_train, columns=['Model', 'Train Accuracy', 'Train Precision', 'Train Recall', 'Train F1 Score', 'Test Accuracy', 'Test Precision', 'Test Recall', 'Test F1 Score'])

print("Results:")
print(comparison_df_train)

In [ ]:
def plot_roc_curve(model_name, model_instance, feature_data, target_data, pastel_color):
    # Predicted probability on data
    y_pred_prob = model_instance.predict_proba(feature_data)[:, 1]

    # Calculating the ROC curve
    fpr, tpr, thresholds = roc_curve(target_data, y_pred_prob)
    roc_auc = auc(fpr, tpr)

    # Display the ROC curve
    plt.figure(figsize=(10, 6))
    plt.plot(fpr, tpr, color=pastel_color, lw=2, label=f'AUC = {roc_auc:.2f}')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'Receiver Operating Characteristic (ROC) Curve - {model_name}')
    plt.legend(loc='lower right')
    plt.show()

    auc_score = roc_auc_score(target_data, y_pred_prob)
    print(f'AUC Score for {model_name}: {auc_score}')

pastel = sns.color_palette("pastel", len(Model))

In [ ]:
for i, (model_name, model_instance) in enumerate(Model):
    plot_roc_curve(model_name, model_instance, feature_validation_scaled, target_validation, pastel[i])

In [ ]:
for i, (model_name, model_instance) in enumerate(Model):
    plot_roc_curve(model_name, model_instance, feature_test_scaled, target_test, pastel[i])

# Model Performance Summary

## Best Performing Model
XGBoost is the best-performing model based on all metrics (accuracy, precision, recall, F1 score, and AUC) for both the training and test sets.

## Overfitting/Underfitting Analysis
- **XGBoost**: The metrics for both training and test sets are very close, indicating no significant overfitting or underfitting. The model generalizes well to unseen data.
- **Decision Tree**: The metrics for training and test sets are also quite close, suggesting no significant overfitting or underfitting. However, its overall performance is lower compared to XGBoost and Random Forest, indicating it might be underperforming.
- **Random Forest**: The metrics for training and test sets are close, indicating good generalization. However, the recall is slightly lower compared to XGBoost, which might suggest a slight room for improvement.

## Hyperparameter Tuning Recommendation
- **XGBoost**: Since it is already the best-performing model, hyperparameter tuning could help to further optimize and enhance its performance.
- **Decision Tree**: Hyperparameter tuning is recommended to improve its performance and potentially bring it closer to that of XGBoost or Random Forest.
- **Random Forest**: Although it performs well, hyperparameter tuning might help improve recall and overall performance.

## Conclusion
XGBoost is currently the best model. Hyperparameter tuning is advisable for XGBoost and Random Forest to further enhance performance, while Decision Tree requires tuning to potentially achieve significant performance improvements.


# Business Recommendations

## 1. Credit Approval Process Optimization
- **Model Integration:** Use the XGBoost model to speed up credit approval and improve accuracy in assessing creditworthiness.
- **Risk Reduction:** Decline high-risk applications and impose stricter terms on medium-risk applicants to reduce non-performing loans (NPLs).

## 2. Personalized Product Offers
- **Customer Segmentation:** Segment customers by risk profile. Offer aggressive credit products to low-risk customers and stricter terms or safer products to high-risk customers.
- **Interest Rate Adjustment:** Provide competitive interest rates for low-risk customers and higher rates for high-risk customers to better price the risk.

## 3. Credit Portfolio Management
- **Regular Monitoring:** Use the model to regularly assess the risk of the existing credit portfolio and take early action on changing risk profiles.
- **Proactive Policies:** Develop proactive policies to manage customers with increasing risk, such as offering loan restructuring programs.

## 4. New Product Development
- **Innovative Credit Products:** Develop new credit products tailored to different risk segments, like microloans for low-risk customers.
- **Credit Insurance:** Offer credit insurance to high-risk customers to mitigate the bank's risk.

## 5. Customer Relationship Management (CRM)
- **Retention Strategies:** Identify low-risk customers at risk of leaving and offer incentives like increased credit limits or loyalty programs to retain them.
- **Proactive Handling:** Proactively manage customers showing signs of financial distress before they default, such as offering loan restructuring or financial counseling.

## Summary
The XGBoost model has proven to be effective in predicting credit risk. By integrating this model into various business processes, the bank can improve risk management, optimize the credit portfolio, enhance customer service. These recommendations not only help in reducing potential losses but also in boosting profitability and business sustainability.
